In [1]:
%load_ext autoreload
%autoreload 2



import swyft.lightning as sl

from lensx import pyrofit_lensing_models as plm
from lensx import swyft_models as sm



import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import matplotlib
from tqdm import tqdm

import sys
sys.path.append('../16-swyft_unet/scripts/')
from plot import *

imkwargs = dict(extent=(-2.5, 2.5, -2.5, 2.5), origin='lower') #left, right, bottom, top
DEVICE = 'cuda'

from predict import Infer, LogPlots

import torch
import swyft
import swyft.lightning as sl

from swyft.lightning import equalize_tensors, RatioSamples



from unet import UNET

In [2]:
n_sub = 3
n_pix = 40
n_msc = 1
grid_low  = torch.tensor([10.0, -2.5, -2.5], device = DEVICE)
grid_high = torch.tensor([11.0, 2.5, 2.5], device = DEVICE)

sigma_n = 0.

MAX_EPCOHS = 10
N_TRAIN = 5_000

In [3]:
model = sm.ModelBlobs(sigma_n = sigma_n, n_pix = n_pix)

In [4]:
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.callbacks import LearningRateMonitor

In [5]:
tag = 'v2'
tbl = pl_loggers.TensorBoardLogger("lightning_logs", name = f'blobs_{tag}')

lr_monitor = LearningRateMonitor(logging_interval="step")
logdir = tbl.experiment.get_logdir()
print("logdir:", logdir)

logdir: lightning_logs/blobs_v2/version_1


# Neural Network 

## (Should go into own `nn/subN/ImgSeg_nn.py` file)

In [6]:
zarr_store = sl.ZarrStore(f'/nfs/scratch/eliasd/lightning/test_me{MAX_EPCOHS}_ntrain{N_TRAIN}_tag{tag}')

zarr_store.init(model = model, N = N_TRAIN, chunk_size = 16)
zarr_store.simulate(model)
store = sl.SampleStore(zarr_store.numpy()) # Load all data into memory

100%|██████████| 10/10 [00:00<00:00, 6306.28it/s]


100%|██████████| 10/10 [00:00<00:00, 6592.74it/s]


100%|██████████| 10/10 [00:00<00:00, 1257.29it/s]


100%|██████████| 10/10 [00:00<00:00, 5896.67it/s]


In [ ]:
class CustomParameterTransform(torch.nn.Module):
    def __init__(self, n_sub, n_pix, n_msc, grid_low, grid_high) -> None:
        super().__init__()
        self.n_sub = n_sub
        self.n_pix = n_pix
        self.n_msc = n_msc
        self.grid_low = grid_low 
        self.grid_high = grid_high
        
    def v_to_grid(self, coord_v):
        coord = torch.clone(coord_v) # Copy otherwise something unexplainable goes wrong

#         coord[:,:,0] = torch.log10(coord[:,:,0]) # Log10 on Mass coordinates  
#         nsub = int(coord.shape[1]/3) # coord.shape = [B, 3*nsub], 3 comes from (x, y, M)
#         low  = torch.tile(self.low, (self.nsub, ))
#         high = torch.tile(self.high, (self.nsub, ))
        return (coord - self.grid_low) / (self.grid_high - self.grid_low)

    def forward(self, coord_v):       
        coord_grid = self.v_to_grid(coord_v)
        
        
        n_batch, n_sub_var, _ =  coord_grid.shape

        z = torch.zeros((n_batch, self.n_msc, self.n_pix, self.n_pix), device = DEVICE)
  
        if not (n_batch == 0 or n_sub_var == 0):
            
            m_grid, x_grid, y_grid = coord_grid.view(-1,3).T.to(DEVICE)            

            x_i = torch.floor((x_grid * self.n_pix).flatten())
            y_i = torch.floor((y_grid * self.n_pix).flatten())
            m_i = torch.floor( m_grid * self.n_msc)
            b_i = torch.repeat_interleave(torch.arange(n_batch, device = DEVICE), n_sub_var)      

            assert (b_i.max(), m_i.max(), y_i.max(), x_i.max()) <= (n_batch, n_msc, n_pix, n_pix)
            
            
            indices = tuple(torch.stack((b_i, m_i, y_i, x_i)).type(torch.long))    
            z[indices] = 1
        
        return z
#         return torch.cat((1-z, z), dim = 1)

class CustomMarginalClassifier(torch.nn.Module):
    def __init__(self, n_marginals: int, n_features: int, 
                 n_sub, n_pix, n_msc, grid_low, grid_high):
        super().__init__()
                
        self.n_marginals = n_marginals
        self.n_features = n_features
        
        out_channels = n_msc*2
        in_channels = 1
        
        self.n_sub = n_sub
        self.n_pix = n_pix
        self.n_msc = n_msc
        self.grid_low = grid_low 
        self.grid_high = grid_high
        
        self.UNet = UNET(out_channels=out_channels, in_channels=in_channels)
                
    def forward(self, sims: torch.Tensor, z: torch.Tensor) -> torch.Tensor:
        
        x = sims
        x = self.UNet(x)

        if not x.is_cuda:
            x = x.to(DEVICE)
        target_map = torch.cat((1-z, z), dim = 1)
        x = x * target_map # B 2*nmc L L
        
        
        x = x.view(x.shape[0], self.n_msc, 2, self.n_pix, self.n_pix)
        x = torch.sum(x, axis = 2)
        
        return x

class RatioEstimatorImgSeg(torch.nn.Module):
    def __init__(self, n_sub, n_pix, n_msc, grid_low, grid_high):
        super().__init__()
        
        _ = 1
        
        self.paramtrans = CustomParameterTransform(n_sub, n_pix, n_msc, grid_low, grid_high)
        self.classifier = CustomMarginalClassifier(_, _,  n_sub, n_pix, n_msc, grid_low, grid_high)

        
    def forward(self, x, z):
#         print('before eq', x.shape, z.shape)
        x, z = equalize_tensors(x, z)
#         print('after  eq', x.shape, z.shape)
        zt = self.paramtrans(z).detach()
        ratios = self.classifier(x, zt)
        w = RatioSamples(z, ratios, metadata = {"type": "ImgSegm"})
        return w


class ImgSegmNetwork(sl.SwyftModule):
    """
    Network to infer log10 cutoff mass from n_obs observations.
    Input parameters:
        - lr: learning rate
        - n_pix: number of pixels of the image
        - n_obs: number of observations to combine
    """

    def __init__(self, n_sub, n_pix, n_msc, grid_low, grid_high):
        super().__init__()
        
        self.n_pix = n_pix
        self.n_msc = n_msc
        
        self.online_z_score = swyft.networks.OnlineDictStandardizingLayer(
            dict(img = (self.n_pix, self.n_pix))
        )
        
        self.classifier = RatioEstimatorImgSeg(
            n_sub, n_pix, n_msc, grid_low, grid_high
        )
        
    def forward(self, x, z):
        x = dict(img = x['img'])
        x = self.online_z_score(x)['img']
        out = self.classifier(x, z['z_sub'])
        return dict(z_pix = out)

    


dataset = sl.SwyftDataModule(
        store=store, model=model, batch_size=16
    )

net = ImgSegmNetwork(n_sub, n_pix, n_msc, grid_low, grid_high)

trainer = sl.SwyftTrainer(
        accelerator="gpu",
        gpus=1,
        max_epochs=MAX_EPCOHS,
        logger=tbl,
        callbacks=[lr_monitor],
    )

final_checkpoint = logdir + "/checkpoint/final.ckpt"
trainer.fit(net, dataset)
trainer.save_checkpoint(final_checkpoint)
trainer.test(net, dataset)
print("saved checkpoint to", final_checkpoint)



GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Set SLURM handle signals.

  | Name           | Type                         | Params
----------------------------------------------------------------
0 | online_z_score | OnlineDictStandardizingLayer | 0     
1 | classifier     | RatioEstimatorImgSeg         | 31.0 M
----------------------------------------------------------------
31.0 M    Trainable params
0         Non-trainable params
31.0 M    Total params
124.146   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/eliasd/.pyenv/versions/3.9.7/envs/lens-3.9.7/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/eliasd/.pyenv/versions/3.9.7/envs/lens-3.9.7/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))
/home/eliasd/.pyenv/versions/3.9.7/envs/lens-3.9.7/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `D

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
post = Infer(n_sub, n_pix, n_msc, grid_low, grid_high)

In [ ]:
posts, targets = post.sample(net, dataset, max_n_test = N_TRAIN)

In [ ]:
LogPlots(tbl, posts, targets, n_alpha = 50);

In [ ]:
assert 1 == 2

In [ ]:
posts.shape, targets.shape

In [ ]:
post = Infer(net, dataset, n_sub, n_pix, n_msc, grid_low, grid_high, max_n_test = 50)

alpha_edges, alpha_centers = post.alpha_edges.cpu(), post.alpha_centers.cpu()
reli_curve = post.get_relicurve().detach().cpu()
post_sum = post.get_post_sum().cpu().detach().numpy()
hist = post.get_histogram().cpu().detach().numpy()


fig = plt.figure(dpi = 200)
plt.step(alpha_centers, reli_curve)
plt.plot((0, 1), (0, 1), 'k:')
# tbl.experiment.add_figure("reli_curve", fig)

fig = plt.figure(dpi = 200)
plt.stairs(hist, alpha_edges, fill=True)
plt.yscale('log')
# tbl.experiment.add_figure("post_all_hist_log", fig)

fig = plt.figure(dpi = 500)
hist, _, _ = plt.hist(post_sum, bins = 50)
plt.plot((n_msc, n_msc), (0, hist.max()), 'k:')
# tbl.experiment.add_figure("post_sum_hist", fig)